# Producer

### Import dependencies

In [16]:
import tweepy
from kafka import KafkaProducer
from json import dumps
import time
import json

### Authentication on Twitter

In [17]:
consumer_key = "dmn51aowr3bYzIWz3ELgARN5x"
consumer_secret = "Q6vHf7FNj1yDvX2vggN6kKLgiDP7t7HL5DNpNvK0akyjzfxTFt"
access_token = "164266780-hcTZEl441WUst50TpiRIZtAd3IHmvPDX5tQUDjqx"
access_token_secret = "AIoEwy9AL52AuHRAKesBph3arofQ4AdLwqYRl8tFnehxd"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

### Connection to Apache Kafka

In [18]:
#Create Kafka Producer

#ip = 'localhost'
ip = '35.228.40.220'
producer = KafkaProducer(bootstrap_servers=[ip+':9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [19]:
#Retrieve TopN Hashtags on the given location

location = 2487956 #san francisco
#location = 2459115 #new york
n = 5 #top n parameter
top_n = [x.get("name") for x in api.trends_place(location)[0].get('trends')[0:n]]
print(top_n)

['Lakers', 'Clippers', '#LoseYouToLoveMe', 'Paul George', 'Pat Bev']


### Pipelined Producer

In [20]:
#Define Listener

class PipelineListener(tweepy.StreamListener):
    def __init__(self,tag,time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        self.tag=tag
        super(PipelineListener, self).__init__()

    def on_status(self, status):
        if (time.time() - self.start_time) < self.limit:
            data = {}
            data['tag'] = self.tag
            data['date'] = str(status.created_at).split(" ")[0]#Keep the date, remove time
            data['tweet'] = status.text
            producer.send("DIC",data)
            print(data)
            return True
        else:
            print("Stopping stream..")
            return False

#Launch listener in series

time_limit = 100

for _ in range(0,5):
    listeners = []
    streams = []
    for i in range(0,n):
        print(top_n[i])
        listeners.append(PipelineListener(top_n[i],time_limit=time_limit))
        streams.append(tweepy.Stream(auth = api.auth, listener=listeners[i]))
        streams[i].filter(track=[top_n[i]], is_async=False, languages=["en"])
        streams[i].disconnect()

Lakers
{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': "RT @Lakers: You know he's been waiting for this 👑💨 https://t.co/eiXev5ysUJ"}
{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': 'RT @Lakers: Season 17. Still zero gravity. 🚀\n\n(📺: TNT) https://t.co/59LhmU4W7z'}
{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': 'RT @Lakers: For our city. #LAL60 https://t.co/6V2yV2Vp5N'}
{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': 'Lakers fans saying “we gonna get better” like they didn’t just lose without Paul George even playing for the clips 🤣'}
{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': 'RT @icecube: Lakers just embarrassed Los Angeles with a half ass effort.'}
{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': 'RT @KhalxdLDN: 2 points between Anthony Davis and Lebron in the 4th. Lakers fans really tried to compare them to Shaq and Kobe 🥴🥴 https://t…'}
{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': 'RT @IAmCoolerCam101: LeBron looking at Kawhi Leonard and the Clippers after the Lakers 

{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': 'RT @NBA: .@KingJames records 18 PTS, 9 REB, 8 AST in the @Lakers season opener against LAC. #KiaTipOff19 #LakeShow https://t.co/Iq4br66ltT'}
{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': 'Woke up and hella ppl hit me talkin junk bout the Lakers-Clips game lol. It’s one game out 82 plus playoffs... lott… https://t.co/OCL9vpeSMH'}
{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': 'Yo @Lakers Maybe you want to sign @carmeloanthony WILL GIVE YOU AT LEAST 18 OFF THE BENCH. SMH come on now'}
{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': "@MyTakes101 @MavsFansForLife you can't spell Los Angeles Lakers without 3 L's 👌"}
{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': 'RT @NBAonTNT: Danny Green (6-6 FG) COULD NOT MISS in the 3rd QTR! 🔥🔥\n\nLakers and Clippers are tied, 85-85 on TNT! https://t.co/bJf4MerCQo'}
{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': "RT @nbastats: Danny Green scored 28 points tonight, the most in a player's @Lakers 

{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': 'RT @JaredDudley619: Best job in the world! Tough loss but a lot to learn from tonight! Will be back on Friday! https://t.co/R9YCyf0PQF'}
{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': "RT @Lakers: You know he's been waiting for this 👑💨 https://t.co/eiXev5ysUJ"}
{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': 'Even tho the lakers lost Bron Bron and AD were hoopin don’t let the 1 loss at the beginning of the season have you… https://t.co/ZG2CQ2lO2w'}
{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': 'RT @fenri1s: #MiGomiluigiFavorito Lakers #LoseYouToLoveMe Clippers #ElPRIqueQueremos Selena #PiamontesaEnTuMesa #Mexico\n#FelizMiercoles #Es…'}
{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': "RT @Lakers: You know he's been waiting for this 👑💨 https://t.co/eiXev5ysUJ"}
{'tag': 'Lakers', 'date': '2019-10-23', 'tweet': 'RT @my2k: I would LOVE to see this shit become a trend\nphotobombing with hong kong solidarity just to force networks to sh

{'tag': 'Clippers', 'date': '2019-10-23', 'tweet': 'RT @raykwong: 😷 Supporters for pro-democracy protesters in Hong Kong were outside Staples Center before the season-opening game between the…'}
{'tag': 'Clippers', 'date': '2019-10-23', 'tweet': 'RT @NBAonTNT: Danny Green (6-6 FG) COULD NOT MISS in the 3rd QTR! 🔥🔥\n\nLakers and Clippers are tied, 85-85 on TNT! https://t.co/bJf4MerCQo'}
{'tag': 'Clippers', 'date': '2019-10-23', 'tweet': 'Conférence Ouest :\n1. Los Angeles Clippers\n2. Los Angeles Lakers \n3. Houston Rockets\n4. Golden State Warriors\n5. Ne… https://t.co/HIrDotgsxc'}
{'tag': 'Clippers', 'date': '2019-10-23', 'tweet': 'RT @MagicJohnson: In the Clippers victory over the Lakers the X factor was the Clippers bench outscoring the Lakers 60-19 and Lou Williams…'}
{'tag': 'Clippers', 'date': '2019-10-23', 'tweet': 'RT @Declinetc: Anyone else see this guy showing his support for #hongkong in the front row of the #Lakers be #Clippers game? Looked like th…'}
{'tag': 'Clippers', 'd

{'tag': '#LoseYouToLoveMe', 'date': '2019-10-23', 'tweet': '♪ I saw the signs and I ignored it ♪\n#LoseYouToLoveMe \n\nI LOVE YOUUU, SG. https://t.co/LGUMKZcMPl'}
{'tag': '#LoseYouToLoveMe', 'date': '2019-10-23', 'tweet': 'RT @musicnewsfact: #LoseYouToLoveMe has now reached the #1 spot on US iTunes. 🇺🇸 🖤\n\nThe song has also went #1 in 30 countries. It did this…'}
{'tag': '#LoseYouToLoveMe', 'date': '2019-10-23', 'tweet': 'RT @fourteencaratss: She looks so angry and irritated at the beginning of the video and in the end she literally looks so relieved and almo…'}
{'tag': '#LoseYouToLoveMe', 'date': '2019-10-23', 'tweet': 'RT @SelenaHQ2: US iTunes 🇺🇸 \n\n1. #LoseYouToLoveMe (+1) https://t.co/rzImllUFI1'}
{'tag': '#LoseYouToLoveMe', 'date': '2019-10-23', 'tweet': 'RT @selenagomez: Thank you all for standing by my side through the highs and lows. I couldn’t do it without you all and I can’t wait to sta…'}
{'tag': '#LoseYouToLoveMe', 'date': '2019-10-23', 'tweet': 'RT @selenagomez: 1 hour.

...

{'tag': '#LoseYouToLoveMe', 'date': '2019-10-23', 'tweet': 'RT @selenagomez: Thank you all for standing by my side through the highs and lows. I couldn’t do it without you all and I can’t wait to sta…'}
{'tag': '#LoseYouToLoveMe', 'date': '2019-10-23', 'tweet': 'RT @itsethancole: Guys I got to listen to this (humble brag) WE ARE GETTING IN OUR FEELINGS TONIGHT AT 9pm let’s gooo https://t.co/NEdtqlpW…'}
{'tag': '#LoseYouToLoveMe', 'date': '2019-10-23', 'tweet': 'RT @selenagomez: Thank you all for standing by my side through the highs and lows. I couldn’t do it without you all and I can’t wait to sta…'}
{'tag': '#LoseYouToLoveMe', 'date': '2019-10-23', 'tweet': 'RT @TSwiftNZ: 🎵 + 👭 | SWIFTIES! Taylor’s BFF @selenagomez’s comeback single #LoseYouToLoveMe is officially out! \n\nLet’s all support it: htt…'}
{'tag': '#LoseYouToLoveMe', 'date': '2019-10-23', 'tweet': 'RT @mainpopgirI: Chanel vs. Walmart\n#LoseYouToLoveMe https://t.co/S23CM2IdJI'}
{'tag': '#LoseYouToLoveMe', 'date': '2019-10-2

{'tag': '#LoseYouToLoveMe', 'date': '2019-10-23', 'tweet': 'RT @AppleMusic: .@selenagomez is back.\nWatch the video for #LoseYoutoLoveMe now. #ShotoniPhone \nhttps://t.co/yj9AWVeFRG https://t.co/UTfJ4b…'}
{'tag': '#LoseYouToLoveMe', 'date': '2019-10-23', 'tweet': 'RT @semiwtv: She lost him but she found herself and somehow that was everything #LoseYouToLoveMe https://t.co/TxFot70JmW'}
{'tag': '#LoseYouToLoveMe', 'date': '2019-10-23', 'tweet': 'RT @OOTFW: "You play stupid games you win stupid prizes" got me thinking. Maybe I shouldn\'t have bought all that Kinder Surprise eggs\n\n#ARI…'}
{'tag': '#LoseYouToLoveMe', 'date': '2019-10-23', 'tweet': 'RT @fuckincruzy: #LoseYouToLoveMe  this dance was killing me softly … F.U heart breaker Love you Queen Selena 🔥🥰 https://t.co/RJZ9O90rrT'}
{'tag': '#LoseYouToLoveMe', 'date': '2019-10-23', 'tweet': 'Adobe AD0-E103 Exam: How to Become a Certified Adobe Developer?\n\n#adobe #LoseYouToLoveMe #exam #certification… https://t.co/1C73VOQaJ9'}
{'tag': 

{'tag': 'Paul George', 'date': '2019-10-23', 'tweet': 'RT @theBSOLZ: So the Clippers beat the Lakers without Paul George and Anthony “who’s gonna guard AD” Davis went 8/21?!?!?! https://t.co/LcK…'}
{'tag': 'Paul George', 'date': '2019-10-23', 'tweet': 'RT @joeylinn_: YOOOOO 😭\n\nPaul George just posted this on Instagram and it might be the best thing I’ve ever seen in my life. https://t.co/8…'}
{'tag': 'Paul George', 'date': '2019-10-23', 'tweet': 'RT @joeylinn_: YOOOOO 😭\n\nPaul George just posted this on Instagram and it might be the best thing I’ve ever seen in my life. https://t.co/8…'}
{'tag': 'Paul George', 'date': '2019-10-23', 'tweet': 'RT @joeylinn_: YOOOOO 😭\n\nPaul George just posted this on Instagram and it might be the best thing I’ve ever seen in my life. https://t.co/8…'}
{'tag': 'Paul George', 'date': '2019-10-23', 'tweet': 'RT @joeylinn_: YOOOOO 😭\n\nPaul George just posted this on Instagram and it might be the best thing I’ve ever seen in my life. https://t.co/8…'}
{'

{'tag': 'Paul George', 'date': '2019-10-23', 'tweet': 'Bruhhhhhh 😂😂'}
{'tag': 'Paul George', 'date': '2019-10-23', 'tweet': '@Lakers lost to the @LAClippers with no Paul george\nLET THAT SINK IN 😂😂😂😂 https://t.co/5XG1GwAVgm'}
{'tag': 'Paul George', 'date': '2019-10-23', 'tweet': 'RT @Ballislife: #TerminatorDarkFate commercial with Kawhi &amp; Paul George!  https://t.co/nScEgIA53H'}
{'tag': 'Paul George', 'date': '2019-10-23', 'tweet': 'and clippers did that without paul george thats why i got clippers winning and lakers need to have bron and ad on t… https://t.co/Pou8sfwqR0'}
{'tag': 'Paul George', 'date': '2019-10-23', 'tweet': 'RT @joeylinn_: YOOOOO 😭\n\nPaul George just posted this on Instagram and it might be the best thing I’ve ever seen in my life. https://t.co/8…'}
{'tag': 'Paul George', 'date': '2019-10-23', 'tweet': 'RT @joeylinn_: YOOOOO 😭\n\nPaul George just posted this on Instagram and it might be the best thing I’ve ever seen in my life. https://t.co/8…'}
{'tag': 'Paul Geo

KeyboardInterrupt: 

### Parallel Producer

In [ ]:
#Define Listener

class ParallelListener(tweepy.StreamListener):
    def __init__(self,time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        super(ParallelListener, self).__init__()

    def on_status(self, status):
        if (time.time() - self.start_time) < self.limit:
            data = {}
            data['date'] = str(status.created_at)
            data['tweet'] = status.text
            json_data = json.dumps(data)
            producer.send("test",json_data)
            print(json_data)
            return True
        else:
            print("Stopping stream..")
            return False

#Launch all listener together

listener = ParallelListener(time_limit=1000)
myStream = tweepy.Stream(auth = api.auth, listener=listener)
myStream.filter(track=top_n, is_async=False, languages=["en"])